In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [ ]:
# Find repo root even if notebook is inside /notebooks
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

dfpath = DATA_DIR / 'combined_data.csv'
df_positive_gen_path = DATA_DIR / 'combined_data_positive_gen.csv'

In [ ]:
def plot_day_distribution(filename,column,xaxis,yaxis,title):
    df =  pd.read_csv(filename, 
                     delimiter=',', 
                     decimal='.', 
                     parse_dates=['ts'], 
                     index_col='ts')
    
    df['15min_period'] = df.index.floor('15min').time

    period_avg = df.groupby('15min_period')[column].mean() # summing over each time of the day 


    ## Plotting
    x_labels = [t.strftime('%H:%M') for t in period_avg.index]

    fig = plt.plot(range(len(period_avg.index)),period_avg.values)

    plt.xticks(range(0, len(period_avg), 4), x_labels[::4], rotation=45)
    plt.xlabel(xaxis, fontsize=12)
    plt.ylabel(yaxis, fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')

    return df, fig


In [ ]:
df_wind, fig_wind = plot_day_distribution(dfpath,'Aircon_WT Power',xaxis='Time of Day (15-minute intervals)',yaxis='Average Power [kW]',title='Yearly Average Power of Aircon Wind Turbine')
plt.show()

In [ ]:
df_pv, fig_pv = plot_day_distribution(dfpath,'PV B715',xaxis='Time of Day (15-minute intervals)',yaxis='Average Power [kW]',title='Yearly Average Power of PV B715')

In [ ]:
df_price, fig_price = plot_day_distribution(dfpath,'DayAheadPriceDKK',xaxis='Time of Day (15-minute intervals)',yaxis='Price on Day Ahead Market in DKK',title='Yearly Average Price on Day Ahead Market')

### Plotting with positive generation only

In [ ]:
df_wind_positive_gen, fig_wind_positive_gen = plot_day_distribution(df_positive_gen_path,'Aircon_WT Power',xaxis='Time of Day (15-minute intervals)',yaxis='Average Power [kW]',title='Yearly Average Power of Aircon Wind Turbine')

In [ ]:
df_wind['Aircon_WT Power'].equals(df_wind_positive_gen['Aircon_WT Power'])

In [ ]:
df_pv_positive_gen, fig_pv_positive_gen = plot_day_distribution(df_positive_gen_path,'PV B715',xaxis='Time of Day (15-minute intervals)',yaxis='Average Power [kW]',title='Yearly Average Power of PV B715')

In [ ]:
df_pv['Aircon_WT Power'].equals(df_pv_positive_gen['Aircon_WT Power'])

In [ ]:
df_price_positive_gen, fig_price_positive_gen = plot_day_distribution(df_positive_gen_path,'DayAheadPriceDKK',xaxis='Time of Day (15-minute intervals)',yaxis='Price on Day Ahead Market in DKK',title='Yearly Average Price on Day Ahead Market')

In [ ]:
df_price['DayAheadPriceDKK'].equals(df_price_positive_gen['DayAheadPriceDKK'])

### Further Analysis on Generation

In [ ]:
pv_df = df_pv_positive_gen['PV B715']
wind_df = df_wind_positive_gen['Aircon_WT Power']

# pv_df.dropna().describe(), wind_df.dropna().describe(); useful for quick metrics


### Comparison with Median and Mean

PV Median: 0.000002 kW --> Power generation from PV is almost negligible most of the time

Wind Median: 0.626148 kW --> Wind generates somewhat reliably 0.6 kW 

In [ ]:
#active pv production 
df_active = df_pv_positive_gen[df_pv_positive_gen['PV B715'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(14,6), sharex = True)

axes[0].hist(df_active["PV B715"], bins=40)
axes[0].set_title("Distribution of Active PV Production")
axes[0].set_xlabel("Power (kW)")
axes[0].set_ylabel("Frequency")

axes[1].hist(df_active['PV B715'])
axes[1].set_yscale("log")
axes[1].set_title("PV Production Distribution (Log Y_axis)")
axes[1].set_xlabel("Power (kW)")
axes[1].set_ylabel("Log Frequency")

plt.tight_layout()
plt.show()



In [ ]:
#Quantiles of pv production 

pv_df.quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])


In [ ]:
## active wind production 

df_active = df_wind_positive_gen[df_wind_positive_gen['Aircon_WT Power'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(14,6), sharex = True)

axes[0].hist(df_active['Aircon_WT Power'], bins=40)
axes[0].set_title("Distribution of Active Wind Production")
axes[0].set_xlabel("Power (kW)")
axes[0].set_ylabel("Frequency")

axes[1].hist(df_active['Aircon_WT Power'])
axes[1].set_yscale("log")
axes[1].set_title("Wind Production Distribution (Log Y_axis)")
axes[1].set_xlabel("Power (kW)")
axes[1].set_ylabel("Log Frequency")

plt.tight_layout()
plt.show()

In [ ]:
#Quantiles of wind production 

wind_df.quantile([0.1, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

### Relation between PV and Solar Irradiance

In [ ]:
## intraday plotting for two variables 

def plot_day_distribution(filename,column1, column2,xaxis,yaxis,title, label1=None, label2=None):
    df =  pd.read_csv(filename, 
                     delimiter=',', 
                     decimal='.', 
                     parse_dates=['ts'], 
                     index_col='ts')
    
    df['15min_period'] = df.index.floor('15min').time

    period_avg = df.groupby('15min_period')[[column1, column2]].mean() # summing over each time of the day 

    ## Labeling

    label1 = label1 if label1 else column1
    label2 = label2 if label2 else column2

    ## Plotting
    x_labels = [t.strftime('%H:%M') for t in period_avg.index]

    fig, ax = plt.subplots(figsize=(10,6))
    
    ax.plot(range(len(period_avg.index)),period_avg[column1].values, label = label1)
    ax.plot(range(len(period_avg.index)),period_avg[column2].values, label = label2)

    ax.set_xticks(range(0, len(period_avg), 4))
    ax.set_xticklabels(x_labels[::4], rotation=45)
    ax.set_xlabel(xaxis, fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend()

    return df, fig

In [ ]:
plot_day_distribution(df_positive_gen_path,'PV B715', '_Meteo715M_MT_INSOLATION_irrad1 | syslab-13/meteo1/INSOLATION_irrad1 | 804142',"Time of Day","Power (kW)","Intraday Profile PV Power and Insolation", label2='Insolation')

In [ ]:
plot_day_distribution(df_positive_gen_path,'Aircon_WT Power', '_Meteo715M_MT_WINDSPEED_wspd1 | syslab-13/meteo1/WINDSPEED_wspd1 | 804146',"Time of Day","Power (kW)","Intraday Profile Wind Power and Wind Speed", label2='Wind Speed')